In [6]:
import pandas as pd
import numpy as np
import scipy.stats as ss

In [7]:
X = [1,2,3,4,5,6,7,8,9,10,11,12,13]
Y = [7,8,8,11,10,11,12,15,20,14,16,15,19]

df_xy = pd.DataFrame({'X': X,'Y': Y})

In [8]:
# use pandas functions to calculate pearson correlation
df_xy.corr('pearson')

,X,Y
X,1.000000,0.887636
Y,0.887636,1.000000


In [9]:
# use pandas functions to calculate spearman correlation
df_xy.corr('spearman')

,X,Y
X,1.000000,0.907594
Y,0.907594,1.000000


In [10]:
import math # for sqrt function

def GetCorrelation_pearson(gvx, gvy):
    t_dxMean = np.mean(gvx)
    t_dyMean = np.mean(gvy)
    
    t_dNominator = 0
    for i in range(len(gvx)):
        t_dNominator += (gvx[i]-t_dxMean)*(gvy[i]-t_dyMean)
    
    t_dDenominator_x = 0
    for i in range(len(gvx)):
        t_dDenominator_x += (gvx[i]-t_dxMean)*(gvx[i]-t_dxMean)
    t_dDenominator_x = math.sqrt(t_dDenominator_x)
    
    t_dDenominator_y = 0
    for i in range(len(gvx)):
        t_dDenominator_y += (gvy[i]-t_dyMean)*(gvy[i]-t_dyMean)
    t_dDenominator_y = math.sqrt(t_dDenominator_y)
    
    return t_dNominator/(t_dDenominator_x*t_dDenominator_y)

In [11]:
# you can use scipy.stats function rankdata() to rank the values in a given vector
def GetCorrelation_spearman(gvx, gvy):
    t_vRank_x = ss.rankdata(gvx)
    t_vRank_y = ss.rankdata(gvy)
    t_vd = t_vRank_x - t_vRank_y
    t_dNominator = 6*sum([d**2 for d in t_vd])
    t_dDenominator = len(gvx)*(len(gvx)**2-1)
    return 1-(t_dNominator/t_dDenominator)

In [12]:
def GetMutualInfo(gdf, sCol1, sCol2):
    
    v_uniqueVals_col1 = gdf[sCol1].unique()
    v_uniqueVals_col2 = gdf[sCol2].unique()
    marginal1 = gdf.groupby(sCol1).size().div(len(gdf))
    marginal2 = gdf.groupby(sCol2).size().div(len(gdf))
    df_condProb = gdf.groupby(sCol1)[sCol2].value_counts() / gdf.groupby(sCol1)[sCol2].count()
    
    cond_prob = {}
    for i in v_uniqueVals_col1:
        for j in v_uniqueVals_col2:
            if (i,j) in df_condProb.index:
                cond_prob[i,j] = df_condProb[i,j]
            else:
                cond_prob[i,j] = 0
    
    joint_prob = {}
    for i in v_uniqueVals_col1:
        for j in v_uniqueVals_col2:
            joint_prob[i,j] = np.round(cond_prob[i,j]*marginal1[i],3)
    
    tmp_mutualInfo = 0
    for i in v_uniqueVals_col1:
        for j in v_uniqueVals_col2:
            if joint_prob[i,j] > 0:
                tmp_mutualInfo += joint_prob[i,j]*math.log(joint_prob[i,j]/(marginal1[i]*marginal2[j]))
    
    return tmp_mutualInfo

In [13]:
GetMutualInfo(df_xy, 'X', 'Y')

2.24828080923372

In [20]:
from sklearn.metrics import mutual_info_score
mutual_info_score(df_xy['X'], df_xy['Y'])

2.2450352741261774

In [14]:
def chi2_by_hand(df, col1, col2):    
    df_cont = pd.crosstab(index = df[col1], columns = df[col2])
    print(df_cont)

    degree_f = (df_cont.shape[0]-1) * (df_cont.shape[1]-1)
    df_cont.loc[:,'Total']= df_cont.sum(axis=1)
    df_cont.loc['Total']= df_cont.sum()
    print('---Observed (O)---')
    print(df_cont)

    df_exp = df_cont.copy()    
    df_exp.iloc[:,:] = np.multiply.outer(
        df_cont.sum(1).values,df_cont.sum().values)/df_cont.sum().sum()            
    print('---Expected (E)---')
    print(df_exp)
        
    # calculate chi-square values
    df_chi2 = ((df_cont - df_exp)**2) / df_exp    
    df_chi2.loc[:,'Total']= df_chi2.sum(axis=1)
    df_chi2.loc['Total']= df_chi2.sum()
    
    print('---Chi-Square---')
    print(df_chi2)
    chi_square_score = df_chi2.iloc[:-1,:-1].sum().sum()
    
    return chi_square_score, degree_f

In [17]:
# chi2_by_hand(df_xy, 'X', 'Y')

from scipy.stats import chisquare
chisquare([16, 18, 16, 14, 12, 12])

Power_divergenceResult(statistic=2.0, pvalue=0.8491450360846096)